# Estruturas Criptográficas - Criptografia e Segurança da Informação

## TP1 - Exercício 1

1. Use a package Cryptography   e  o package ascon (instalar daqui) para  criar um comunicação privada assíncrona em modo  “Lightweight Cryptography” entre um agente Emitter e um agente Receiver que cubra os seguintes aspectos:
    1. Autenticação do criptograma e dos metadados (associated data) usando Ascon (ver implementação aqui) em modo de cifra.
    2. As chaves de cifra, autenticação  e  os “nounces” são gerados por um gerador pseudo aleatório (PRG)  usando o Ascon em modo XOF. As diferentes chaves para inicialização do PRG são inputs do emissor e do receptor.
    3. Para implementar a comunicação cliente-servidor use o package python `asyncio`.

## Resolução

Para resolver este exercício comecamos por instalar e importar os packages necessarios.

Este setup inicial envolveu, para além do referido no exercício, a utilização do package `nest_asyncio` para garantir a execução apropriada do `asyncio` no `Jupyter Notebook`. 

In [13]:
%pip install ascon asyncio nest_asyncio

import ascon
import random
import asyncio
import hashlib
import nest_asyncio

Note: you may need to restart the kernel to use updated packages.


Para continuar o setup do ambiente iniciamos o `nest_asyn` e as variáveis globais do programa.
Estas incluem o tamanho utilizado para a geração das hashs e as tags `tag` e `ext` para a geração da `associated data`.

In [14]:
nest_asyncio.apply()  
hashlength=16         # Tamanho da hash

Função para calcular a hash de uma mensagem. Isto será utilizado como associated data para verificar a autenticidade das mensagens.

In [15]:

def calculate_sha256(message):
    if isinstance(message, str):
        message = message.encode()

    sha256_hash = hashlib.sha256(message).hexdigest()
    return sha256_hash

O próximo passo foi a definição da função para cifrar mensagens. 

Para tal seguiram-se os seguintes passos:

- Criar a `associated data` para a mensagem especifica
- Criar o `nounce` utilizando `Ascon-XOF`
- Cifrar a mensagem utilizando a chave, o `nounce` e a `associated data`

In [16]:
def cipher_message(in_message,key):
    associated_data=calculate_sha256(in_message).encode()
    nounce_seed=str(random.getrandbits(128))
    nounce=ascon.hash(nounce_seed.encode(),variant="Ascon-Xof", hashlength=hashlength)
    try:
        out_message=ascon.encrypt(key, nounce, associated_data, in_message.encode(), variant="Ascon-128")
        print(f"Sending: {in_message}")
        print(f"Outgoing >>> ({out_message},{nounce},{associated_data})")
    except Exception as e:
        print(e)
    return (out_message,nounce,associated_data)

Similarmente foi criada a função para decifrar mensagens. 

Esta segue um processo semelhante:

- Criar a `associated data` para a mensagem esperada
- Decifrar a mensagem utilizando a chave, o `nounce` recebido e a `associated data`
- Verificar que a mensagem foi decifrada com sucesso

In [39]:
def read_message(text,key,nounce,associated_data):
    print(f"Incoming <<< ({text},{nounce},{associated_data})")
    try:
        out_message=ascon.decrypt(key, nounce, associated_data, text, variant="Ascon-128")
    except Exception as e:
        return "[ERROR] Message could not be decrypted"
    if out_message==None and calculate_sha256(out_message.decode())!=associated_data.decode():
        return "[ERROR] Message has been tampered"
    return out_message.decode()

Com as funções de cifrar e decifrar criadas segue-se a função para emitir as mensagens cifradas. 

Esta segue os seguintes passos:

- Aguarda `input` do utilizador para nova mensagem
- Cifrar a mensagem com recurso à chave e à função previamente criada
- Adicionar a mensagem (e o respetivo `nounce`) à queue
- Repetir até que seja inserido o código para terminar pelo utilizador ("exit") 

In [18]:
async def emitter(queue,key):
    loop = asyncio.get_event_loop()
    while True:
        message=await loop.run_in_executor(None, input, "Message to send > ")
        out=cipher_message(message,key)
        await queue.put(out)
        if message=="exit":
            break

A função para receber as mensagens cifradas, para além de as decifrar, verifica a sua integridade e se o _nounce_ associado já foi recebido previamente.

Para tal segue o seguintes processo:

- Aguarda que uma nova mensagem seja obtida da queue
- Verificar se o `nounce` já foi recebido previamente
    - Se sim, rejeitar a mensagem
- Decifrar a mensagem com recurso à chave, ao `nounce` e à função previamente criada
- Repetir até que seja recebido um código para terminar pelo emissor ("exit") 

In [19]:
async def receiver(queue,key):
    known_nounces=[]
    while True:
        message,nounce,associated_data=await queue.get()
        if nounce in known_nounces:
            print("Repeated nounce, ignoring message")
        else:
            known_nounces.append(nounce)
            text=read_message(message,key,nounce,associated_data)
            print(f"Received: {text}")
        if text=="exit":
            break

Finalmente a função principal do programa foi criada com a seguinte estrutura:

- Criar a queue onde serão transmitidas mensagens
- Criar uma key com recurso a `Ascon-XOF` com base numa seed fornecida pelo utilizador
- Criar e lançar os processos para o emissor e recetor

In [20]:
async def main():
    queue = asyncio.Queue()
    key_seed=input("Seed for key > ")
    key=ascon.hash(key_seed.encode(),variant="Ascon-Xof", hashlength=hashlength)
    print(f"Session Key: {key}")
    # Create separate tasks for emitter and receiver
    emitter_task = asyncio.create_task(emitter(queue, key))
    receiver_task = asyncio.create_task(receiver(queue, key))
    # Wait for both tasks to complete
    await asyncio.gather(emitter_task, receiver_task)
    print("All tasks completed")


Finalmente o programa pode ser testado:

In [21]:
await main()

Session Key: b'\xb0\xbdX,\xc1\x00\x19\xfd~c\x1d\x888\xdbzN'
Sending: hello
Outgoing >>> (b'a\xf2\xa7\xaa\xd8+\x890\xab\x004\xb1\x8e\xea\xad9\xfb"\xc0B\xcf',b'O\x0cK\x0c\xcc]\x02(\xaf%\x8dXp!\xa5\x81',b'2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824')
Incoming <<< (b'a\xf2\xa7\xaa\xd8+\x890\xab\x004\xb1\x8e\xea\xad9\xfb"\xc0B\xcf',b'O\x0cK\x0c\xcc]\x02(\xaf%\x8dXp!\xa5\x81',b'2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824')
Received: hello
Sending: nice
Outgoing >>> (b'\xbcJ\x89\xcf-\x98\xa5\x10d\xe2\xe2B\x1b\xf6\xe9\xcf\x85\xdbH\xfd',b'] 7\r\xa7\x82\xf1\xa7\xe5\xaae\xc3\xe8\x1b\xe1L',b'e186022d0931afe9fe0690857e32f85e50165e7fbe0966d49609ef1981f920c6')
Incoming <<< (b'\xbcJ\x89\xcf-\x98\xa5\x10d\xe2\xe2B\x1b\xf6\xe9\xcf\x85\xdbH\xfd',b'] 7\r\xa7\x82\xf1\xa7\xe5\xaae\xc3\xe8\x1b\xe1L',b'e186022d0931afe9fe0690857e32f85e50165e7fbe0966d49609ef1981f920c6')
Received: nice
Sending: exit
Outgoing >>> (b'\x17.zZ*7E\xb9\x96\x86\xe7\xbf\xde\xac\xa0\xfc]M\x80\x90

__Possiveis Problemas:__

- Alterações à mensagem enviada 
- Retransmissões de mensagens com o mesmo `nounce`

Para testar estes possiveis problemas foram adicionados os comandos `altered_test` e `repeat_test` à função `cipher_message()` para simular, respetivamente, estas atitudes.

In [37]:
sent_nounces=[]

def cipher_message(in_message,key):
    associated_data=calculate_sha256(in_message).encode()
    # Message is repeated
    if in_message=="repeat_test":
        nounce=sent_nounces[-1]
    else:
        nounce_seed=str(random.getrandbits(128))
        nounce=ascon.hash(nounce_seed.encode(),variant="Ascon-Xof", hashlength=hashlength)

    try:
        out_message=ascon.encrypt(key, nounce, associated_data, in_message.encode(), variant="Ascon-128")
        print(f"Sending: {in_message}")

        # Message data is altered
        if in_message=="altered_test":
            print(f"Original >>> ({out_message},{nounce},{associated_data})")
            out_message=out_message[:2]+(f'{out_message[2]+1}'.encode())+out_message[3:]

        print(f"Outgoing >>> ({out_message},{nounce},{associated_data})")
        sent_nounces.append(nounce)
    except Exception as e:
        print(e)
    
    return (out_message,nounce,associated_data)

Assim podemos testar estas atitudes iniciando o programa e, após o envio da primeira mensagem, introduzir o comando pretendido:

- `altered_test` => Para enviar uma mensagem com a `associated_data` alterada
- `repeat_test` => Para enviar uma mensagem com o `nounce` anterior 

In [40]:
await main()

Session Key: b'O\x1fM\xe3LSB\x18\xac\x91\xe2\xb3A\x90&0'
Original >>> (b'\xfb\x87\x8eC8\x99\xca\xcf\xfd\xe24\xe4\xa8\xa3\x11Dmf\x03{\xce\xe38b\x01U\x93A',b'\x80\xfc\x9c\xe0\x05j/\xaf"B~\xcdce*\xcb',b'4db0985a79cdcb5f0ae754067b4f6316812e19b52edb3c833ae81250e81b1c53')
Sending: altered_test
Outgoing >>> (b'\xfb\x87143C8\x99\xca\xcf\xfd\xe24\xe4\xa8\xa3\x11Dmf\x03{\xce\xe38b\x01U\x93A',b'\x80\xfc\x9c\xe0\x05j/\xaf"B~\xcdce*\xcb',b'4db0985a79cdcb5f0ae754067b4f6316812e19b52edb3c833ae81250e81b1c53')
Incoming <<< (b'\xfb\x87143C8\x99\xca\xcf\xfd\xe24\xe4\xa8\xa3\x11Dmf\x03{\xce\xe38b\x01U\x93A',b'\x80\xfc\x9c\xe0\x05j/\xaf"B~\xcdce*\xcb',b'4db0985a79cdcb5f0ae754067b4f6316812e19b52edb3c833ae81250e81b1c53')
Received: [ERROR] Message could not be decrypted
Sending: exit
Outgoing >>> (b'v2?\rI2\x12\x18\x8d\xd1\xfe\xe4\rr\xba\x182\x88\xf0\xe0',b'T\xacV}\xa5\x90\xb5y\x9e\xec\x18\xa4\x1b\x9cD\xcd',b'e596899f114b5162402325dfb31fdaa792fabed718628336cc7a35a24f38eaa9')
Incoming <<< (b'v2?\rI2\x12\x18\x8d